In [9]:
import pandas as pd
df = pd.read_csv('iphone.csv')
df.head(2)

,Product Name,Product URL,Brand,Sale Price,Mrp,Number Of Ratings,Number Of Reviews,Upc,Star Rating,Ram
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",https://www.flipkart.com/apple-iphone-8-plus-g...,Apple,49900,49900,3431,356,MOBEXRGV7EHHTGUH,4.6,2 GB
1,"APPLE iPhone 8 Plus (Space Grey, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,Apple,84900,84900,3431,356,MOBEXRGVAC6TJT4F,4.6,2 GB


#### 1 - The column names have spaces . rename the column names to have underscore '_' instead of space

In [11]:
df.columns = df.columns.str.replace(' ', '_')
df.head(2)

,Product_Name,Product_URL,Brand,Sale_Price,Mrp,Number_Of_Ratings,Number_Of_Reviews,Upc,Star_Rating,Ram
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",https://www.flipkart.com/apple-iphone-8-plus-g...,Apple,49900,49900,3431,356,MOBEXRGV7EHHTGUH,4.6,2 GB
1,"APPLE iPhone 8 Plus (Space Grey, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,Apple,84900,84900,3431,356,MOBEXRGVAC6TJT4F,4.6,2 GB


#### 2- start rating for some of the models is missing in the dataset. fill those missing values with the average rating all the models.

In [48]:
#2- start rating for some of the models is missing in the dataset. 
#fill those missing values with the average rating all the models.
# Convert to numeric, setting errors='coerce' turns invalid strings into NaN
df['Star_Rating'] = pd.to_numeric(df['Star_Rating'], errors='coerce')

# Calculate average excluding NaNs
Avg_Rating = df['Star_Rating'].mean()

# Fill missing values with the average
df['Star_Rating'].fillna(Avg_Rating, inplace=True)

# Display the dataframe
df.head(2)

,Product_Name,Product_URL,Brand,Sale_Price,Mrp,Number_Of_Ratings,Number_Of_Reviews,Upc,Star_Rating,Ram,Discount_Percentage,Color,iPhone Version
0,"APPLE iPhone 8 Plus (Gold, 64 GB)",https://www.flipkart.com/apple-iphone-8-plus-g...,Apple,49900,49900,3431,356,MOBEXRGV7EHHTGUH,4.6,2 GB,0.0,Gold,iPhone 8
1,"APPLE iPhone 8 Plus (Space Grey, 256 GB)",https://www.flipkart.com/apple-iphone-8-plus-s...,Apple,84900,84900,3431,356,MOBEXRGVAC6TJT4F,4.6,2 GB,0.0,Space Grey,iPhone 8


#### 3- Now instead of filling missing values with avg rating of full dataset , fill with avg rating based on RAM.  Example :  if rating for a 2 gb phone is missing then take average of all other 2 gb phones rating and fill that value. 

In [18]:
# Calculate the average rating for each RAM group
average_ratings_by_ram = df.groupby('Ram')['Star_Rating'].mean().reset_index()

# Rename the 'Star Rating' column to 'Average Rating' for clarity
average_ratings_by_ram.rename(columns={'Star_Rating': 'Average Rating'}, inplace=True)

# Perform a left join to fill missing ratings based on RAM
df = df.merge(average_ratings_by_ram, on='Ram', how='left')

# Fill missing ratings with the calculated average ratings
df['Star_Rating'].fillna(df['Average Rating'], inplace=True)

# Drop the 'Average Rating' column if not needed
df.drop(columns=['Average Rating'], inplace=True)

# Display the modified DataFrame
print(df)

                                Product_Name   
0          APPLE iPhone 8 Plus (Gold, 64 GB)  \
1   APPLE iPhone 8 Plus (Space Grey, 256 GB)   
2       APPLE iPhone 8 Plus (Silver, 256 GB)   
3            APPLE iPhone 8 (Silver, 256 GB)   
4              APPLE iPhone 8 (Gold, 256 GB)   
..                                       ...   
57            APPLE iPhone SE (Black, 64 GB)   
58           APPLE iPhone 11 (Purple, 64 GB)   
59            APPLE iPhone 11 (White, 64 GB)   
60            APPLE iPhone 11 (Black, 64 GB)   
61              APPLE iPhone 11 (Red, 64 GB)   

                                          Product_URL  Brand  Sale_Price   
0   https://www.flipkart.com/apple-iphone-8-plus-g...  Apple       49900  \
1   https://www.flipkart.com/apple-iphone-8-plus-s...  Apple       84900   
2   https://www.flipkart.com/apple-iphone-8-plus-s...  Apple       84900   
3   https://www.flipkart.com/apple-iphone-8-silver...  Apple       77000   
4   https://www.flipkart.com/apple-iphone-8

#### 4- create a new column in the dataframe "Discount_Percentage" based on MRP and sale value

In [49]:
df['Discount_Percentage'] = ((df['Mrp'] - df['Sale_Price']) / df['Mrp']) * 100
df['Discount_Percentage']

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
        ...    
57    24.814536
58    14.391621
59    14.391621
60    14.391621
61    14.391621
Name: Discount_Percentage, Length: 62, dtype: float64

#### 5- which model has highest percent discount ?

In [50]:
sorted_df = df.sort_values(by='Discount_Percentage', ascending=False)

# Get the model with the highest discount and its corresponding percentage
model_with_highest_discount = sorted_df.iloc[0]['Product_Name']
highest_discount_percentage = sorted_df.iloc[0]['Discount_Percentage']

model_with_highest_discount
#highest_discount_percentage

'APPLE iPhone 11 Pro (Midnight Green, 64 GB)'

#### 6- find total no of models  each space configuration (128 GB , 64 GB etc)

In [32]:
space_configuration_counts = df['Ram'].value_counts()

# Rename the columns for clarity
space_configuration_counts.columns = ['Space Configuration', 'Total Models']

# Display the result
print(space_configuration_counts)

Ram
4 GB    29
6 GB    19
2 GB    13
3 GB     1
Name: count, dtype: int64


#### 7- find total number of models for each color 

In [34]:
# Define a function to extract color based on a pattern

def extract_color(product_name):
    parts = product_name.split('(')     # Split the product name by '(' and ')' to get parts within parentheses
    if len(parts) > 1:                  # Check if the second part contains a comma (,) to identify color
        color_part = parts[1].strip()
        if ',' in color_part:
            color = color_part.split(',')[0].strip()
            return color
    return None

# Apply the extract_color function to create a 'Color' column
df['Color'] = df['Product_Name'].apply(extract_color)

# Count the number of models for each color
color_counts = df['Color'].value_counts().reset_index()

# Rename the columns for clarity
color_counts.columns = ['Color', 'Total Models']

# Display the result
print(color_counts)

             Color  Total Models
0            Black            10
1            White            10
2           Silver             7
3             Gold             6
4       Space Grey             6
5              Red             5
6   Midnight Green             4
7     Pacific Blue             4
8         Graphite             4
9             Blue             2
10             RED             1
11           Coral             1
12           Green             1
13          Purple             1


#### 8- find total number of models by iphone version : 
eg :- 
iphone 8:  9, 
iphone XR : 5

In [36]:
# Define a function to extract iPhone version
def extract_iphone_version(product_name):
    parts = product_name.split('iPhone')        # Split the product name by 'iPhone' and get the part after 'iPhone'
    if len(parts) > 1:
        version = parts[1].strip().split(' ')[0]
        return 'iPhone ' + version
    return None

# Apply the extract_iphone_version function to create an 'iPhone Version' column
df['iPhone Version'] = df['Product_Name'].apply(extract_iphone_version)

# Count the number of models for each iPhone version
iphone_version_counts = df['iPhone Version'].value_counts().reset_index()

# Rename the columns for clarity
iphone_version_counts.columns = ['iPhone Version', 'Total Models']

# Display the result
print(iphone_version_counts)


  iPhone Version  Total Models
0      iPhone 12            26
1      iPhone 11            16
2       iPhone 8             8
3      iPhone SE             6
4      iPhone XR             5
5      iPhone XS             1


#### 9- list top 5 models having highest no of reviews 

In [38]:
# Sort the DataFrame by 'Number Of Reviews' in descending order
df_sorted = df.sort_values(by='Number_Of_Reviews', ascending=False)

# Select the top 5 models with the highest number of reviews
top_5_models = df_sorted.head(5)

# Display the result
print(top_5_models)

                       Product_Name   
23  Apple iPhone SE (White, 256 GB)  \
53  APPLE iPhone SE (Black, 128 GB)   
55    APPLE iPhone SE (Red, 128 GB)   
57   APPLE iPhone SE (Black, 64 GB)   
52   APPLE iPhone SE (White, 64 GB)   

                                          Product_URL  Brand  Sale_Price   
23  https://www.flipkart.com/apple-iphone-se-white...  Apple       44999  \
53  https://www.flipkart.com/apple-iphone-se-black...  Apple       34999   
55  https://www.flipkart.com/apple-iphone-se-red-1...  Apple       34999   
57  https://www.flipkart.com/apple-iphone-se-black...  Apple       29999   
52  https://www.flipkart.com/apple-iphone-se-white...  Apple       29999   

      Mrp  Number_Of_Ratings  Number_Of_Reviews               Upc   
23  54900              95909               8161  MOBFRFXHPZCHAPEH  \
53  44900              95909               8161  MOBFWQ6BHUEVZPXD   
55  44900              95909               8161  MOBFWQ6BJTVFKPEJ   
57  39900              95909    

#### 10 - what is the price diffrence between highest price and lowest price iphone (based on mrp)

In [40]:
# Find the highest and lowest MRP values
highest_mrp = df['Mrp'].max()
lowest_mrp = df['Mrp'].min()

# Calculate the price difference
price_difference = highest_mrp - lowest_mrp
price_difference

110000

#### 11 - find total no of reviews for iphone 11 and iphone 12 category . Output should have only 2 rows (for 11 and 12).

In [42]:
# Filter the DataFrame for iPhone 11 and iPhone 12 categories
iphone_11_reviews = df[df['Product_Name'].str.contains('iPhone 11', case=False)]
iphone_12_reviews = df[df['Product_Name'].str.contains('iPhone 12', case=False)]

# Calculate the total number of reviews for each category
total_reviews_11 = iphone_11_reviews['Number_Of_Reviews'].sum()
total_reviews_12 = iphone_12_reviews['Number_Of_Reviews'].sum()

# Create a new DataFrame with the results
result_df = pd.DataFrame({
    'iPhone Category': ['iPhone 11', 'iPhone 12'],
    'Total Reviews': [total_reviews_11, total_reviews_12]
})

result_df

,iPhone Category,Total Reviews
0,iPhone 11,25965
1,iPhone 12,2208


#### 12- which iphone has 3rd highest MRP

In [44]:
# Sort the DataFrame by 'Mrp' column in descending order and reset the index
sorted_df = df.sort_values(by='Mrp', ascending=False).reset_index(drop=True)

# Get the iPhone with the 3rd highest MRP
third_highest_mrp_iphone = sorted_df.loc[2]

# Print the details of the iPhone with the 3rd highest MRP
print(third_highest_mrp_iphone)

Product_Name                APPLE iPhone 11 Pro (Midnight Green, 512 GB)
Product_URL            https://www.flipkart.com/apple-iphone-11-pro-m...
Brand                                                              Apple
Sale_Price                                                        117900
Mrp                                                               140300
Number_Of_Ratings                                                   7088
Number_Of_Reviews                                                    523
Upc                                                     MOBFKCTSSJCWYGCC
Star_Rating                                                          4.6
Ram                                                                 4 GB
Discount_Percentage                                            15.965788
Color                                                     Midnight Green
iPhone Version                                                 iPhone 11
Name: 2, dtype: object


#### 13- what is the average mrp of iphones which costs above 100,000

In [46]:
# Filter the DataFrame to include only iPhones with MRP above 100,000
high_mrp_iphones = df[df['Mrp'] > 100000]

# Calculate the average MRP for the filtered iPhones
average_mrp = high_mrp_iphones['Mrp'].mean()

# Print the average MRP
print(average_mrp)

130559.09090909091


#### 14- which iphone with 128 GB space has highest ratings to review ratio

In [ ]:
# Filter the DataFrame to include only iPhones with 128 GB of space
iphone_128gb = df[df['Ram'] == '128 GB']

# Calculate the ratings-to-reviews ratio for each iPhone
iphone_128gb['Ratings to Reviews Ratio'] = iphone_128gb['Number Of Ratings'] / iphone_128gb['Number Of Reviews']

# Find the iPhone with the highest ratings-to-reviews ratio
highest_ratio_iphone = iphone_128gb.loc[iphone_128gb['Ratings to Reviews Ratio'].idxmax()]